# Collaborativ Filtering

Ziel ist, eine Tabelle zu haben, mit verschiedenen Playlisten in den Zeilen und Songs in den Spalten. Die Zellen sagen aus, ob ein Song in einer Playlist ist oder nicht (0 oder 1).

Die Tabelle kann dann für Collab Filtering verwendet werden.

Da unsere Datnebank in einem anderen Netzwerk liegt, wird zunächst ein Tunnel über ein Terminal benötigt:

```ssh <username>@login1.mi.hs-rm.de -L 9001:db.intern.mi.hs-rm.de:5432```

In [1]:
import psycopg2
import pandas as pd
from getpass import getpass
from sklearn.metrics.pairwise import cosine_similarity

## Generate Collab Filtering Table

Also brauchen wir zunächst einmal die Tabelle mit dem Inhalt, welches Lied in welcher Playlist ist.

### Datenbankverbindung

In [2]:
user = input("Bitte User für DB eingeben")
pswd = getpass("Bitte User für DB eingeben")
conn = psycopg2.connect(f"dbname='orent001_spotify_test' user='{user}' host='localhost' port='9001' password='{pswd}'")

In [3]:
# query = "SELECT playlist.name AS playlist, song.track_name AS song FROM playlist INNER JOIN p_enthaelt_s ON p_enthaelt_s.playlist = playlist.playlist_id INNER JOIN song ON song.song_id = p_enthaelt_s.song;"
# playlists = pd.io.sql.read_sql_query(query,conn)
# playlists

### Select Data from SQL and pivot it to a collab usefull table

pd.pivot_table macht aus:           gleich:

```
playlist | song | drinnen               song | A | B | C 
---------+------+--------           playlist |   |   |   
 0       | A    | 1                 ---------+---+---+---
 0       | B    | 1          ==>     0       | 1 | 1 | 1 
 0       | C    | 1                  1       | 0 | 1 | 0 
 1       | B    | 1                  2       | 0 | 1 | 1 
 2       | B    | 1      
 2       | C    | 1      
```

Dann werden die Column Namen entwernt und die Werte in der Tabelle standartisiert sodass wir dann sowas haben:
```
         |  A    |  B   |  C 
---------+-------+------+-------
 0       |  0    | 0    |  0 
 1       | -0.33 | 0.67 | -0.33 
 2       | -0.67 | 0.33 |  0.33 
 ```

dauert mit den Testdaten ca. 2 bis 5 Minuten

In [4]:
query = "SELECT playlist, song, 1 as drinnen FROM p_enthaelt_s;"
songs = pd.io.sql.read_sql_query(query,conn)
table = pd.pivot_table(songs, values='drinnen', columns='song', index='playlist')

# um unregemmaessigkeiten zu entfernen, werden alle Songs, die in weniger als 10 Playlisten drinnen sind, rausgeschmissen.
# table = table.dropna(thresh=10, axis=0).fillna(0)

table = table.fillna(0)

table.columns.name = None

def standardize(row):
    new_row = (row - row.mean())/1
    return new_row

ratings_std = table.apply(standardize)
ratings_std

Da wir Item to Item collab filtering wollen (und nicht playlist to playlist), wollen wir jetzt unsere recommendation Matrix erstellen, die wie folget gefildet wird:

```
         |  A    |  B   |  C                  | A | B | C 
---------+-------+------+-------           ---+---+---+---
 0       |  0    | 0    |  0        ==>     A | 1 | ? | ?
 1       | -0.33 | 0.67 | -0.33             B | ? | 1 | ?
 2       | -0.67 | 0.33 |  0.33             C | ? | ? | 1  
 ```
Wert für das Beispiel hab ich nicht ausgerechnet.

Dafür verwende ich die cosine_similarity function von pandas.

Dauer ca. 17 Minuten bei der Test Datenmenge :(

In [ ]:
item_similarity = cosine_similarity(table.T)
item_similarity_df = pd.DataFrame(item_similarity, table.columns, table.columns)
item_similarity_df

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/mi/orent001/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-7dad6ba72af6>", line 1, in <module>
    item_similarity = cosine_similarity(table.T)
  File "/usr/lib/python3/dist-packages/sklearn/metrics/pairwise.py", line 917, in cosine_similarity
    X, Y = check_pairwise_arrays(X, Y)
  File "/usr/lib/python3/dist-packages/sklearn/metrics/pairwise.py", line 107, in check_pairwise_arrays
    warn_on_dtype=warn_on_dtype, estimator=estimator)
  File "/usr/lib/python3/dist-packages/sklearn/utils/validation.py", line 453, in check_array
    _assert_all_finite(array)
  File "/usr/lib/python3/dist-packages/sklearn/utils/validation.py", line 44, in _assert_all_finite
    " or a value too large for %r." % X.dtype)
ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

During handling of the above 

In [ ]:
item_similarity_df.to_csv('item_similarity.csv', index=False)

In [ ]:
def get_similar_songs(songURL):
    similar_score = item_similarity_df[songURL]*0.5
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

def get_similar_playlist(songs):
    similar_songs = pd.DataFrame()
    for song in songs:
        similar_songs = similar_songs.append(get_similar_songs(song), ignore_index=True)
    similar_songs = similar_songs.sum().sort_values(ascending=False)
    return similar_songs

# get_similar_songs(['spotify:track:7zzSan8uETSRwOsg2CDFpN'], ['spotify:track:7zwGkKY304rsu1rYPH0zvI'])
